# Ciclos Políticos do Ceará

**Trabalho de Conclusão de Curso — MBA em Jornalismo de Dados (IDP)** 
**Responsável:** Igor Cavalcante — victorigor.ac@gmail.com  
**Tema:** Análise dos ciclos políticos do Ceará ao longo de 30 anos  
**Etapa:** 1/3 — Processamento de dados [coleta, limpeza e padronização]  
**Data:** 12/2025  

Este notebook é o primeiro de três que analisam os ciclos políticos no Ceará nas últimas três décadas. A partir do desempenho partidário, a pesquisa deve mapear as legendas que crescentam e reduziram no Estado, e fornecer dados para uma reportagem que explique o que há por trás dessas oscilações.  

Para tanto, serão considerados os desempenhos partidários de prefeitos, deputados estaduais e federais. No caso dos mandatários municipais, eles revelam quem controla a política local e os recursos municipais. Já os deputados estaduais consolidam o apoio político ao Governo do Estado. E os deputados federais mostram como o Ceará se insere nas dinâmicas nacionais dos partidos.  

Neste primeiro momento, os dados são coletados a partir do [portal de Dados Abertos do Tribunal Superior Eleitoral](https://dadosabertos.tse.jus.br/) e padronizados para posterior análise.  

Para a pesquisa, foram considerados os dados de Resultado das eleições municipais e gerais entre 1994 e 2024, totalizando 16 pleitos.

# 1. Processamento

Nesta etapa, os arquivos originais do TSE são coletados e limpos para garantir a consistência e a comparabilidade das informações que serão analisadas no projeto.

### Configuração inicial

A partir daqui, são definidos os diretórios, o padrão de codificação e as colunas de interesse para que toda a análise tenha a mesma formatação.

In [2]:
# Biblioteca, caminhos, bloco de leituras e seleção
import os
import re
import pandas as pd
from pathlib import Path

pasta_base = r"C:\Python\eleicoes_ceara"
pasta_brutos = fr"{pasta_base}\br_dados_brutos"
arquivo_saida = fr"{pasta_base}\dados_nacionais.csv"

CHUNK = 500_000

encoding = ["latin-1", "utf-8"]

colunas_selecionadas = [
    "ANO_ELEICAO","SG_UF","SG_PARTIDO","DS_CARGO","DS_SIT_TOT_TURNO",
    "SQ_CANDIDATO","NR_CANDIDATO","NM_CANDIDATO","CD_MUNICIPIO",
    "QT_VOTOS","NR_TURNO"
]

re_ano = re.compile(r"(\d{4})")

### Funções

As funções automatizam tarefas de padronização, correção de inconsistências e leitura em blocos.

In [3]:
# Padronização e limpeza de nomes, tipos e estrutura das colunas
def padronizar_colunas(df: pd.DataFrame) -> pd.DataFrame:
    
    df.columns = df.columns.str.upper().str.strip()

    # Renomeia colunas equivalentes
    renomes = {}
    for nome_coluna in ["NM_URNA_CANDIDATO", "NM_VOTAVEL", "NOME_CANDIDATO"]:
        if nome_coluna in df.columns:
            renomes[nome_coluna] = "NM_CANDIDATO"
            break

    if "QT_VOTOS_NOMINAIS" in df.columns and "QT_VOTOS" not in df.columns:
        renomes["QT_VOTOS_NOMINAIS"] = "QT_VOTOS"

    if renomes:
        df = df.rename(columns=renomes)

    for c in colunas_selecionadas:
        if c not in df.columns:
            df[c] = pd.NA

    # Padroniza textos
    for col in df.columns:
        try:
            df[col] = df[col].astype(str).str.upper().str.strip()
        except Exception:          
            continue

    if "QT_VOTOS" in df.columns:
        df["QT_VOTOS"] = (
            pd.to_numeric(df["QT_VOTOS"], errors="coerce")
            .fillna(0)
            .astype(int)
        )

    return df[colunas_selecionadas]

In [4]:
# Localização e listagem dos arquivos brutos do TSE (ordenados por ano)
def arquivos_brutos(pasta_raiz: Path):

    arquivos = []
    for raiz, _dirs, files in os.walk(pasta_raiz):
        for f in files:
            nome = f.lower()
            if nome.startswith("votacao_candidato_munzona_") and nome.endswith("_brasil.csv"):
                caminho = Path(raiz) / f
                m = re_ano.search(f)
                ano = int(m.group(1)) if m else 0
                arquivos.append((ano, caminho))
    return [c for _, c in sorted(arquivos, key=lambda x: x[0])]

In [5]:
# Leitura em blocos e unificação de arquivos
def ler_gravar(caminho_arquivo: Path, saida: Path, header: bool):

    for enc in encoding:
        try:
            for chunk in pd.read_csv(
                caminho_arquivo,
                sep=";",
                dtype=str,
                chunksize=CHUNK,
                low_memory=False,
                on_bad_lines="skip",
                encoding=enc
            ):
                chunk = padronizar_colunas(chunk)
                mode = "w" if header else "a"
                chunk.to_csv(
                    saida,
                    sep=";",
                    index=False,
                    encoding="utf-8",
                    mode=mode,
                    header=header
                )
                header = False

            print(f"OK: {caminho_arquivo.name} (encoding: {enc})")
            return
        except UnicodeDecodeError:
            continue

    print(f"AVISO: não foi possível decodificar {caminho_arquivo.name}. Pulando...")

# 2. Leitura e unificação da base de dados

Esta etapa executa as funções criadas anteriormente, integrando os arquivos eleitorais de diferentes anos e cargos em um único dataset consolidado, uma base limpa e padronizada, pronto para análise.

In [6]:
# Leitura e padronização dos arquivos. Em seguida, consolidação dos dados em um único arquivo
arquivos = arquivos_brutos(Path(pasta_brutos))
arquivo_saida = Path(arquivo_saida)

if not arquivos:
    print(f"Nenhum arquivo *_BRASIL.csv encontrado em {Path(pasta_brutos).resolve()}")
else:
    if arquivo_saida.exists():
        arquivo_saida.unlink()

    header = True
    for caminho in arquivos:
        caminho = Path(caminho)
        print(f"Processando: {caminho}")
        ler_gravar(caminho, arquivo_saida, header=header)
        header = False

    if arquivo_saida.exists():
        print(f"\n Consolidado criado em: {arquivo_saida}")
        preview = pd.read_csv(arquivo_saida, sep=";", nrows=5, dtype=str, encoding="utf-8")
        display(preview)
    else:
        print("Consolidado não foi criado.")

Processando: C:\Python\eleicoes_ceara\br_dados_brutos\gerais\VOTACAO_CANDIDATO_1994_mun_zon\votacao_candidato_munzona_1994_BRASIL.csv
OK: votacao_candidato_munzona_1994_BRASIL.csv (encoding: latin-1)
Processando: C:\Python\eleicoes_ceara\br_dados_brutos\gerais\VOTACAO_CANDIDATO_1998_mun_zon\votacao_candidato_munzona_1998_BRASIL.csv
OK: votacao_candidato_munzona_1998_BRASIL.csv (encoding: latin-1)
Processando: C:\Python\eleicoes_ceara\br_dados_brutos\municipais\VOTACAO_CANDIDATO_2000_mun_zon\votacao_candidato_munzona_2000_BRASIL.csv
OK: votacao_candidato_munzona_2000_BRASIL.csv (encoding: latin-1)
Processando: C:\Python\eleicoes_ceara\br_dados_brutos\gerais\VOTACAO_CANDIDATO_2002_mun_zon\votacao_candidato_munzona_2002_BRASIL.csv
OK: votacao_candidato_munzona_2002_BRASIL.csv (encoding: latin-1)
Processando: C:\Python\eleicoes_ceara\br_dados_brutos\municipais\VOTACAO_CANDIDATO_2004_mun_zon\votacao_candidato_munzona_2004_BRASIL.csv
OK: votacao_candidato_munzona_2004_BRASIL.csv (encoding: l

,ANO_ELEICAO,SG_UF,SG_PARTIDO,DS_CARGO,DS_SIT_TOT_TURNO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_CANDIDATO.1,CD_MUNICIPIO,QT_VOTOS,NR_TURNO
0,1994,AC,PRN,PRESIDENTE,NÃO ELEITO,28010003611655,36,CARLOS ANTONIO GOMES,CARLOS ANTONIO GOMES,01120,0,1
1,1994,AC,PRONA,PRESIDENTE,NÃO ELEITO,28010005611657,56,ENEAS FERREIRA CARNEIRO,ENEAS FERREIRA CARNEIRO,01120,0,1
2,1994,AC,PPR,PRESIDENTE,NÃO ELEITO,28010001111650,11,ESPERIDIAO AMIN HELOU FILHO,ESPERIDIAO AMIN HELOU FILHO,01120,0,1
3,1994,AC,PSDB,PRESIDENTE,ELEITO,28010004511656,45,FERNANDO HENRIQUE CARDOSO,FERNANDO HENRIQUE CARDOSO,01120,0,1
4,1994,AC,PSC,PRESIDENTE,NÃO ELEITO,28010002011654,20,HERNANI GOULART FORTUNA,HERNANI GOULART FORTUNA,01120,0,1


### Ajuste de partidos renomeados

In [7]:
# Siglas que passaram por mudanças de nome ao longo do tempo
nomes = {
    "PMDB": "MDB*",
    "MDB": "MDB*",
    "PPB": "PP*",
    "PP": "PP*",
    "PT DO B": "AVANTE*",
    "AVANTE": "AVANTE*",
    "PRB": "REPUBLICANOS*",
    "REPUBLICANOS": "REPUBLICANOS*",
    "PPS": "CIDADANIA*",
    "CIDADANIA": "CIDADANIA*",
    "PEN": "PATRIOTA*",
    "PATRIOTA": "PATRIOTA*",
    "PTRB": "PRTB*",
    "PRTB": "PRTB*",
    "PSDC": "DC*",
    "DC": "DC*",
    "PR": "PL*",
    "PL": "PL*",
    "PSN": "PHS*",
    "PHS": "PHS*",
    "PRN": "PTC*",
    "PTC": "AGIR*",
    "AGIR": "AGIR*",
    "PFL": "DEM*",
    "DEM": "DEM*",
    "PMR": "PRB*",
    "SD": "SOLIDARIEDADE*",
    "SOLIDARIEDADE": "SOLIDARIEDADE*",
    "PMN": "MOBILIZA*",
    "MOBILIZA": "MOBILIZA*"
}


In [8]:
df = pd.read_csv(pasta_base + "/dados_nacionais.csv", sep=";", encoding="utf-8")

In [9]:
# Atualização das legendas renomeadas
df["SG_PARTIDO"] = df["SG_PARTIDO"].replace(nomes)

In [10]:
df.to_csv(pasta_base + "/dados_nacionais.csv", sep=";", encoding="utf-8")